# * Neural Network
## 5. 출력층 설계하기

> - 출력층 활성화 함수는 **풀고자 하는 문제의 성질에 맞게** 정의
- i.e. 회귀 - 항등함수, 2클래스 분류 - 시그모이드함수, 다중 분류 - 소프트맥스함수

___



### 1) 항등 함수와 소프트맥스 함수 구현하기

- 항등 함수(identity function)은 입력을 그대로 출력 $\sigma()$

<br>
- 소프트맥스 함수(softmax function)는 아래의 식과 같음
$$
y_k = \frac{exp(a_k)}{\sum_{i=1}^{n}exp(a_i)}
$$

- 여기서 $n$은 출력층의 뉴런 수, $k$는 $k$번째 출력임을 의미
- 소프트맥스 함수의 분자는 입력신호 $a_k$의 지수함수, 분모는 모든 입력신호의 지수함수의 합으로 구성
- 다시 이야기 하면, **'활성화되기 전의 출력값들을 전체와의 비율로 나타내주는 활성화 함수'**
- 아래 그림 참조
![](image/fig 3-22.png)



- 소프트맥스 함수를 한 단계씩 구현하면 아래와 같음

In [1]:
a = np.array([.3, 2.9, 4])
exp_a = np.exp(a)
print(exp_a)


NameError: name 'np' is not defined

In [ ]:
sum_exp_a = exp_a.sum()
print(sum_exp_a)

In [ ]:
y = exp_a / sum_exp_a
print(y)

- 위 내용을 파이썬 함수로 정의 시 다음과 같이 정의 가능

In [ ]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = exp_a.sum()
    y = exp_a / sum_exp_a
    
    return y

___
### 2) 소프트맥스 함수 구현 시 주의점

- 위의 softmax 함수엔 오버플로우의 위험이 있음
- softmax 함수는 지수함수를 사용하는 만큼 매우 큰 값을 반환하고, 때로는 inf 값을 반환하기도 함
- 큰 수 간의 나눗셈으로 인해 수치가 '불안정'해 질 수도 있음
- i.e. $e^{10}$은 20,000이 넘고, $e^{100}$만 되어도 0이 40개가 넘는 큰 값이 됨

- 이를 해결하기 위해 softmax 함수 구현을 다음과 같이 개선 가능

$$
y_k = \frac{exp(a_k)}{\sum_{i = 1}^nexp(a_i)} = \frac{Cexp(a_k)}{C\sum_{i=1}^nexp(a_i)}\\
= \frac{exp(a_k + logC)}{\sum_{i=1}^nexp(a_i + logC)}\\
= \frac{exp(a_k+C')}{\sum_{i=1}^nexp(a_i+C')}
$$

1. 첫 번째 변형에서는 $C$라는 임의의 정수는 분자와 분모 양쪽에 곱함
- $C$를 지수함수 안으로 옮겨 $logC$로 변환
- $logC$를 $C'$로 치환


- $C'$에 어떤 값을 대입해도 상관 없지만, **입력신호 중 최댓값**을 이용하여 오버플로우를 막는 것이 일반적

In [ ]:
a = np.array([1010, 1000, 990])
np.exp(a) / np.exp(a).sum()

In [ ]:
c = np.max(a)
a - c

In [ ]:
np.exp(a-c) / np.exp(a-c).sum()

- 위의 예시처럼 아무런 조치 없이 입력값에 큰 값이 들어가게 된다면 오버플로우로 인해 nan이 출력됨
- 하지만 입력 신호 중 최댓값을 빼주는 것만으로 올바르게 구현 가능
- 아래는 softmax 함수를 수정한 버전임

In [2]:
def softmax(a):
    c = a.max()               #오버플로우 방지
    exp_a = np.exp(a-c)
    sum_exp_a = exp_a.sum()
    y = exp_a / sum_exp_a
    
    return y

In [ ]:
print(a)
softmax(a)

___
### 3) 소프트맥스 함수의 특징

In [4]:
a = np.array([.3, 2.9, 4])
y = softmax(a)
print(y)
np.sum(y)

[0.01821127 0.24519181 0.73659691]


1.0

- softmax 함수의 출력은 0에서 1사이의 실수이며, **총합은 1**임
- 총합이 1이라는 성질 덕분에 softmax 함수의 출력을 **확률**로 해석할 수 있음
- 앞의 예에서 y[2]의 확률이 73.7%로 가장 높으니 답이 2번째 클래스일 가능성이 높다고 결론 내릴 수 있음

- 신경망 분류에서는 일반적으로 가장 큰 출력을 내는 뉴런에 해당하는 클래스로만 인식
- 따라서 출력층의 소프트맥스 함수는 생략하는 편 (자원낭비 방지)
___

### 4) 출력층의 뉴런 수 정하기
- 출력층의 뉴런 수는 풀려는 문제에 맞게 설정
- 분류에서는 분류하고 싶은 클래스 수로 설정
![](image/fig 3-23.png)